<center>
<img src="http://www.dallard.tech/file/logo1.png" style="float:left; max-width: 120px; display: inline" alt="Dallard Consulting"/> 
</center>

# Apprentissage Statistique / Machine avec <a href="https://www.python.org/"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f8/Python_logo_and_wordmark.svg/390px-Python_logo_and_wordmark.svg.png" style="max-width: 150px; display: inline" alt="Python"/></a> & <a href="http://scikit-learn.org/stable/#"><img src="http://scikit-learn.org/stable/_static/scikit-learn-logo-small.png" style="max-width: 180px; display: inline" alt="Scikit-Learn"/></a>
**Petit résumé**: Ce notebook introduit l'utilisation de la librairie `scikit-learn` pour la modélisation et l'apprentissage. Pourquoi utiliser `scikit-learn` ? Ou non ? Liste des fonctionnalités, quelques exemples de mise en oeuvre de modélisation régression logistique, $k$-plus proches voisins, arbres de décision, forêts aléatoires.On vera aussi l'optimisation des paramètres (complexité) des modèles par validation croisée, les fontions de chaînage (*pipeline*) de transformations et estimations. D'autres fonctionalités de `Scikit-learn` sont abordées dans les prochains cours. 


**Avertissement :** Ce notebook est plutôt technique, pas de problème si vous ne comprenez pas tout directement dès la première lecture prenez le temps de le lire 'oklm' !  

## 1 Introduction
### 1.1 Scikit-learn basic 
L'objectif de ce notebook est d'introduire l'utilisation de la librairie `scikit-learn` de Python. Seule l'utilisation directe des fonctions de modélisation sont abordées. Petit rappel générique de `scikit-learn`. 

- Cette librairie manipule des objets de classe `array` de `numpy` *chargés en mémoire* et donc de taille limitée par la RAM de l'ordinateur; de façon analogue R charge en RAM des objets de type `data.frame`.
- `Scikit-learn` reconnaît quelque fois la classe `DataFrame` de `pandas`. Une variable binaire est simplement remplacée par un codage *(0,1)* mais, en présence de plusieurs modalités, traiter celles-ci comme des entiers n'a pas de sens statistique et remplacer une variable qualitative par l'ensemble des indicatrices (*dummy variables (0,1)*) de ses modalités  complique les stratégies de sélection de modèle tout en rendant inexploitable l'interprétation statistique. 
- Les implémentations en Python de certains algorithmes dans `scikit-learn` sont souvent rapide et utilisent implicitement les capacités de parallélisation.


### 1.2 Fonctions d'apprentissage de Scikit-learn
La communauté qui développe cette librairie est très active et la fait évoluer très rapidement.  Ne pas hésiter à consulter la [documentation](http://scikit-learn.org/stable/user_guide.html) pour des compléments. Voici une sélection de ses principales fonctionnalités en lien avec la modélisation : 

- Transformations (standardisation, discrétisation binaire, regroupement de modalités, imputations rudimentaires de données manquantes) , "vectorisation" de corpus de textes (encodage, catalogue, Tf-idf), images;
- Modéle linéaire général avec pénalisation (ridge, lasso, elastic net...), analyse discriminante linéaire et quadratique,  $k$ plus proches voisins,  processus gaussiens, classifieur bayésien naïf, arbres de régression et classification (CART), agrégation de modèles (bagging, random forest, adaboost, gradient tree boosting), perceptron multicouche (réseau de neurones), SVM (classification, régression, détection d'atypiques...);
- Algorithmes de validation croisée (loo, k-fold, VC stratifiée...) et sélection de modèles, optimisation sur une grille de paramètres, séparation aléatoire apprentissage et test, courbe ROC;
- Enchaînement (*pipeline*) de traitements.

En résumé, cette librairie est focalisée sur les aspects "machine" de l'apprentissage de données quantitatives (séries, signaux, images) volumineuses. 

### 1.3 Objectif 
Illustrer la mise en oeuvre de quelques fonctionnalités ainsi que apprendre à consulter la [documentation](http://scikit-learn.org/stable/user_guide.html) et ses nombreux [exemples](http://scikit-learn.org/stable/auto_examples/index.html) pour plus de détails sur les possibilités d'utilisation de `scikit-learn`. 

Deux jeux de données élémentaires sont utilisés, des variables explicatives qualitatives et quantitatives généralement dans un objet de la classe `DataFrame`. Pour être utilisé dans `scikit-learn` les données doivent souvent être transformées en un objet de classe `Array` de `numpy` par le  remplacement des variables qualitatives par les indicatrices de leurs modalités.  On peut aussi noterun autre ensemble de données, lui entièrement quantitatif. C'est un problème classique et simplifié de [reconnaissance de caractères](http://archive.ics.uci.edu/ml/datasets/Pen-Based+Recognition+of+Handwritten+Digits) qui est inclus dans la librairie `scikit-learn`.


## 2 Extraction des échantillons
Le travail préliminaire consiste à séparer les échantillons en une partie *apprentissage* et une autre de *test* pour estimer sans biais l'[erreur de prévision](http://wikistat.fr/pdf/st-m-app-risque-estim.pdf). L'optimisation (biais-variance) de la complexité des modèles est réalisée en minimisant l'erreur estimée par [validation croisée](http://wikistat.fr/pdf/st-m-app-risque-estim.pdf) $V-fold$. 

### 2.1 Données "Caractères"
Elles sont disponibles dans la librairie `Scikit-learn`.

In [2]:
# Importations 
import matplotlib.pyplot as plt
from sklearn import datasets
%matplotlib inline
# les données
digits = datasets.load_digits()
# Contenu et mode d'obtention
print(digits)

{'data': array([[  0.,   0.,   5., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,  10.,   0.,   0.],
       [  0.,   0.,   0., ...,  16.,   9.,   0.],
       ..., 
       [  0.,   0.,   1., ...,   6.,   0.,   0.],
       [  0.,   0.,   2., ...,  12.,   0.,   0.],
       [  0.,   0.,  10., ...,  12.,   1.,   0.]]), 'target': array([0, 1, 2, ..., 8, 9, 8]), 'target_names': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), 'images': array([[[  0.,   0.,   5., ...,   1.,   0.,   0.],
        [  0.,   0.,  13., ...,  15.,   5.,   0.],
        [  0.,   3.,  15., ...,  11.,   8.,   0.],
        ..., 
        [  0.,   4.,  11., ...,  12.,   7.,   0.],
        [  0.,   2.,  14., ...,  12.,   0.,   0.],
        [  0.,   0.,   6., ...,   0.,   0.,   0.]],

       [[  0.,   0.,   0., ...,   5.,   0.,   0.],
        [  0.,   0.,   0., ...,   9.,   0.,   0.],
        [  0.,   0.,   3., ...,   6.,   0.,   0.],
        ..., 
        [  0.,   0.,   1., ...,   6.,   0.,   0.],
        [  0.,   0.,   1., ..

In [ ]:
images_and_labels = list(zip(digits.images, 
   digits.target))
for index, (image, label) in  enumerate(images_and_labels[:8]):
     plt.subplot(2, 4, index + 1)
     plt.axis('off')
     plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
     plt.title('Training: %i' % label)

In [ ]:
# variables prédictives et cible
X=digits.data
y=digits.target

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=11)

### 2.2 Données "Titanic"

Les données sur le naufrage du Titanic sont décrites dans le calepin consacré à la librairie *pandas*. Reconstruire la table des données en lisant le fichier .csv.

In [14]:
# Lire les données d'apprentissage
import pandas as pd
path=''  # si les données sont déjà dans le répertoire courant SINON metter le bon chemin 
# path='http://www.math.univ-toulouse.fr/~besse/Wikistat/data/'
df=pd.read_csv(path+'titanic-train.csv',skiprows=1,header=None,usecols=[1,2,4,5,9,11],
  names=["Surv","Classe","Genre","Age","Prix","Port"],dtype={"Surv":object,"Classe":object,"Genre":object,"Port":object})
df.head()

,Surv,Classe,Genre,Age,Prix,Port
0,0,3,male,22.0,7.2500,S
1,1,1,female,38.0,71.2833,C
2,1,3,female,26.0,7.9250,S
3,1,1,female,35.0,53.1000,S
4,0,3,male,35.0,8.0500,S


In [15]:
df.shape # dimensions
# Redéfinir les types 
df["Surv"]=pd.Categorical(df["Surv"],ordered=False)
df["Classe"]=pd.Categorical(df["Classe"],ordered=False)
df["Genre"]=pd.Categorical(df["Genre"],ordered=False)
df["Port"]=pd.Categorical(df["Port"],ordered=False)
df.dtypes

Surv      category
Classe    category
Genre     category
Age        float64
Prix       float64
Port      category
dtype: object

In [16]:
df.count()

Surv      891
Classe    891
Genre     891
Age       714
Prix      891
Port      889
dtype: int64

In [17]:
# imputation des valeurs manquantes
df["Age"]=df["Age"].fillna(df["Age"].median())
df.Port=df["Port"].fillna("S")

In [18]:
# Discrétiser les variables quantitatives
df["AgeQ"]=pd.qcut(df.Age,3,labels=["Ag1","Ag2","Ag3"])
df["PrixQ"]=pd.qcut(df.Prix,3,labels=["Pr1","Pr2","Pr3"])
# redéfinir les noms des modalités 
df["Surv"]=df["Surv"].cat.rename_categories(["Vnon","Voui"])
df["Classe"]=df["Classe"].cat.rename_categories(["Cl1","Cl2","Cl3"])
df["Genre"]=df["Genre"].cat.rename_categories(["Gfem","Gmas"])
df["Port"]=df["Port"].cat.rename_categories(["Pc","Pq","Ps"])
df.head()

,Surv,Classe,Genre,Age,Prix,Port,AgeQ,PrixQ
0,Vnon,Cl3,Gmas,22.0,7.2500,Ps,Ag1,Pr1
1,Voui,Cl1,Gfem,38.0,71.2833,Pc,Ag3,Pr3
2,Voui,Cl3,Gfem,26.0,7.9250,Ps,Ag2,Pr1
3,Voui,Cl1,Gfem,35.0,53.1000,Ps,Ag3,Pr3
4,Vnon,Cl3,Gmas,35.0,8.0500,Ps,Ag3,Pr1


In [20]:
#vérification 
df.count()

Surv      891
Classe    891
Genre     891
Age       891
Prix      891
Port      891
AgeQ      891
PrixQ     891
dtype: int64

### 2.3 Construction des échantillons d'entrainement et de test 

In [23]:
# Construction des indicatrices
df_q=df.drop(["Age","Prix"],axis=1)
df_q.head()
# Indicatrices
dc=pd.DataFrame(pd.get_dummies(df_q[["Surv","Classe","Genre","Port","AgeQ","PrixQ"]]))
dc.head()

,Surv_Vnon,Surv_Voui,Classe_Cl1,Classe_Cl2,Classe_Cl3,Genre_Gfem,Genre_Gmas,Port_Pc,Port_Pq,Port_Ps,AgeQ_Ag1,AgeQ_Ag2,AgeQ_Ag3,PrixQ_Pr1,PrixQ_Pr2,PrixQ_Pr3
0,1,0,0,0,1,0,1,0,0,1,1,0,0,1,0,0
1,0,1,1,0,0,1,0,1,0,0,0,0,1,0,0,1
2,0,1,0,0,1,1,0,0,0,1,0,1,0,1,0,0
3,0,1,1,0,0,1,0,0,0,1,0,0,1,0,0,1
4,1,0,0,0,1,0,1,0,0,1,0,0,1,1,0,0


In [24]:
# Table des indicatrices
df1=pd.get_dummies(df_q[["Surv","Classe","Genre","Port","AgeQ","PrixQ"]])
# Une seule indicatrice par variable binaire
df1=df1.drop(["Surv_Vnon","Genre_Gmas"],axis=1)
# Variables quantitatives
df2=df[["Age","Prix"]]
# Concaténation
df_c=pd.concat([df1,df2],axis=1)
# Vérification
df_c.columns

Index(['Surv_Voui', 'Classe_Cl1', 'Classe_Cl2', 'Classe_Cl3', 'Genre_Gfem',
       'Port_Pc', 'Port_Pq', 'Port_Ps', 'AgeQ_Ag1', 'AgeQ_Ag2', 'AgeQ_Ag3',
       'PrixQ_Pr1', 'PrixQ_Pr2', 'PrixQ_Pr3', 'Age', 'Prix'],
      dtype='object')